In [1]:
# 모듈 import
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
import wandb
import copy

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
# class name 정의
classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

model_name = 'cascade_swin_t'

# config file 들고오기
cfg = Config.fromfile('./_cascade_rcnn/cascade_swin_t_fpn_3x_coco.py')
root='/opt/ml/detection/dataset/'

# config 수정
cfg.data.train.classes = classes
cfg.data.train.img_prefix = root 
cfg.data.train.ann_file = './stratified_kfold/cv_train_1.json' 


cfg.data.val.classes = classes
cfg.data.val.img_prefix = root
cfg.data.val.ann_file = './stratified_kfold/cv_val_1.json' 

cfg.data.test.classes = classes
cfg.data.test.img_prefix = root
cfg.data.test.ann_file = root + 'test.json'

cfg.data.samples_per_gpu = 6

cfg.seed = 2021
cfg.gpu_ids = [0]
cfg.work_dir = './work_dirs/' + model_name

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.checkpoint_config = dict(max_keep_ckpts=1, interval=1)

In [4]:
# datasets
datasets = [build_dataset(cfg.data.train)]
datasets

loading annotations into memory...
Done (t=0.25s)
creating index...
index created!


[
 CocoDataset Train dataset with number of images 8673, and instance counts: 
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | category          | count | category      | count | category        | count | category    | count | category     | count |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+
 | 0 [General trash] | 3737  | 1 [Paper]     | 5660  | 2 [Paper pack]  | 1070  | 3 [Metal]   | 1098  | 4 [Glass]    | 920   |
 | 5 [Plastic]       | 3044  | 6 [Styrofoam] | 1205  | 7 [Plastic bag] | 5779  | 8 [Battery] | 154   | 9 [Clothing] | 582   |
 +-------------------+-------+---------------+-------+-----------------+-------+-------------+-------+--------------+-------+]

In [ ]:
# model
model = build_detector(cfg.model)
model.init_weights()

In [ ]:
# training
train_detector(model, datasets, cfg, distributed=False, validate=True) 